<a href="https://colab.research.google.com/github/ElizabethA01/RNN-Play-Generator/blob/main/RNN_Play_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RNN Play Generator**

I used RNN to generate a play. I showed the RNN an example of something I wanted to recreate and it will learn how to write a version of its own. 

I created this using a character predictive model that will take a variable length sequence as input and predict the next character. 

In [1]:
# imports 
%tensorflow_version 2.x 
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
# Dataset. download the file

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')



1130496/1115394 [==============================] - 0s 0us/step


In [3]:
# how to load your own file

# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

In [4]:
# read content of files

#read, then decode for py 2 compat
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('length of text: {} characters'.format(len(text)))

length of text: 1115394 characters


In [5]:
# take a look at the first 250 characters in text

print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# encoding

# need to encode each line as a different integer.

vocab = sorted(set(text))

#creating a mapping from unique characters to indices

char2idx = {u:i for i, u in enumerate(vocab)}

idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
# lets look at how part of our text is encoded

print('Text:', text[:13])
print('Encoded:', text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [8]:
# function that converts our numeric values to text

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [9]:
# creating training examples

# pick a sequence length

seq_length = 100 #length of sequency for a training example
examples_per_epoch = len(text)//(seq_length+1)

#creating training exmaples/ targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


In [10]:
# turn the stream into batches of desired length

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


In [11]:
# split sequences of length 101 into input and output

def split_input_target(chunk): 
  input_text = chunk[:-1] # hell
  target_text = chunk[1:] # ello
  return input_text, target_text

dataset = sequences.map(split_input_target) # use map to apply the above function to every entry

In [12]:
for x, y in dataset.take(2):
  print('\n\nEXAMPLE\n')
  print('INPUT')
  print(int_to_text(x))
  print('\nOUPUT')
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [13]:
# make the training batches

BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # vocab is number of unique numbers
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# **Building the model**

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                                         batch_input_shape=[batch_size, None]),
                               tf.keras.layers.LSTM(rnn_units,
                                                    return_sequences=True,
                                                    stateful=True,
                                                    recurrent_initializer='glorot_uniform'),
                               tf.keras.layers.Dense(vocab_size)
  ])
  return model

model=build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


# **Creating a Loss function**

In [15]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch) # ask our model for a prediction on our first batch of training data
  print(example_batch_predictions.shape, '(batch_size, sequence_length, vocab_size)') # print out the output shape


(64, 100, 65) (batch_size, sequence_length, vocab_size)


In [16]:
# the prediction is an array of 64 arrays, one for each entry in the batch

print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-9.56329459e-05  6.22043619e-03 -5.05873526e-04 ... -3.22831329e-03
   -1.28869235e-03  2.70821201e-03]
  [ 4.52523492e-03  3.52056045e-03 -2.88971845e-04 ... -6.77891122e-03
    1.76084228e-03 -1.49698101e-03]
  [ 4.75468906e-03  2.52831541e-03  5.13948093e-04 ... -3.07715381e-03
    2.82810396e-03  2.98377126e-03]
  ...
  [ 1.24665201e-02  1.39832229e-03 -8.43041483e-03 ... -2.52279919e-03
   -3.16232070e-03 -8.37589090e-04]
  [ 1.32429749e-02 -6.60178531e-03 -7.32389791e-03 ... -2.58095493e-03
   -8.68461933e-03  5.00956783e-03]
  [ 1.37334270e-02 -2.96925241e-03 -6.16753614e-03 ...  3.19698156e-04
   -3.01624252e-03  5.21255331e-03]]

 [[ 2.77484627e-03  2.63897306e-03  1.49044651e-03 ...  1.07557466e-03
    4.93761105e-03  1.98380463e-03]
  [ 4.47391160e-03 -1.51268672e-03  1.24489819e-03 ...  1.39713602e-03
   -5.29685430e-03  3.32107418e-03]
  [ 3.37505736e-03 -2.73080054e-03  3.61397141e-03 ...  6.28639711e-03
   -2.05111527e-03  3.90712777e-03]
  ...
  [ 6.473

In [17]:
# let's examine one prediction
pred = example_batch_predictions
print(len(pred))
print(pred)

64
tf.Tensor(
[[[-9.56329459e-05  6.22043619e-03 -5.05873526e-04 ... -3.22831329e-03
   -1.28869235e-03  2.70821201e-03]
  [ 4.52523492e-03  3.52056045e-03 -2.88971845e-04 ... -6.77891122e-03
    1.76084228e-03 -1.49698101e-03]
  [ 4.75468906e-03  2.52831541e-03  5.13948093e-04 ... -3.07715381e-03
    2.82810396e-03  2.98377126e-03]
  ...
  [ 1.24665201e-02  1.39832229e-03 -8.43041483e-03 ... -2.52279919e-03
   -3.16232070e-03 -8.37589090e-04]
  [ 1.32429749e-02 -6.60178531e-03 -7.32389791e-03 ... -2.58095493e-03
   -8.68461933e-03  5.00956783e-03]
  [ 1.37334270e-02 -2.96925241e-03 -6.16753614e-03 ...  3.19698156e-04
   -3.01624252e-03  5.21255331e-03]]

 [[ 2.77484627e-03  2.63897306e-03  1.49044651e-03 ...  1.07557466e-03
    4.93761105e-03  1.98380463e-03]
  [ 4.47391160e-03 -1.51268672e-03  1.24489819e-03 ...  1.39713602e-03
   -5.29685430e-03  3.32107418e-03]
  [ 3.37505736e-03 -2.73080054e-03  3.61397141e-03 ...  6.28639711e-03
   -2.05111527e-03  3.90712777e-03]
  ...
  [ 6.473

In [18]:
# prediction at first timestep
time_step = pred[0]
print(len(time_step))
print(time_step)


100
tf.Tensor(
[[-9.5632946e-05  6.2204362e-03 -5.0587353e-04 ... -3.2283133e-03
  -1.2886924e-03  2.7082120e-03]
 [ 4.5252349e-03  3.5205605e-03 -2.8897185e-04 ... -6.7789112e-03
   1.7608423e-03 -1.4969810e-03]
 [ 4.7546891e-03  2.5283154e-03  5.1394809e-04 ... -3.0771538e-03
   2.8281040e-03  2.9837713e-03]
 ...
 [ 1.2466520e-02  1.3983223e-03 -8.4304148e-03 ... -2.5227992e-03
  -3.1623207e-03 -8.3758909e-04]
 [ 1.3242975e-02 -6.6017853e-03 -7.3238979e-03 ... -2.5809549e-03
  -8.6846193e-03  5.0095678e-03]
 [ 1.3733427e-02 -2.9692524e-03 -6.1675361e-03 ...  3.1969816e-04
  -3.0162425e-03  5.2125533e-03]], shape=(100, 65), dtype=float32)


In [ ]:
# to determine the predicted character we need to sample the output distribution (pick a value based on probability)
sampled_indices = tf.random.categorical(pred, num_samples=1)

#reshape the array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sample_indices, (1,-1))[0]
predicted_chars = int_to_text(sample_indices)

predicted_chars # this is what the model predicted for training sequence 


In [21]:
# create a loss function that can compare the output to the expected output and give us some numeric value representing how close they were

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True) # built in keras loss function

# **Compiling the model**

In [22]:
model.compile(optimizer='adam', loss=loss)

In [23]:
# directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# **Train the model**

In [24]:
history = model.fit(data, epochs = 10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 43s 227ms/step - loss: 2.7800
Epoch 2/10
172/172 [==============================] - 40s 227ms/step - loss: 2.1030
Epoch 3/10
172/172 [==============================] - 41s 228ms/step - loss: 1.8396
Epoch 4/10
172/172 [==============================] - 41s 227ms/step - loss: 1.6756
Epoch 5/10
172/172 [==============================] - 40s 227ms/step - loss: 1.5720
Epoch 6/10
172/172 [==============================] - 41s 227ms/step - loss: 1.5018
Epoch 7/10
172/172 [==============================] - 41s 227ms/step - loss: 1.4525
Epoch 8/10
172/172 [==============================] - 40s 226ms/step - loss: 1.4148
Epoch 9/10
172/172 [==============================] - 40s 226ms/step - loss: 1.3829
Epoch 10/10
172/172 [==============================] - 40s 226ms/step - loss: 1.3562


# **Load the model**

In [25]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [26]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint('./training_checkpoints/ckpt_' + str(checkpoint_num)))
model.build(tf.TensorShape([1,None]))

In [28]:
def generate_text(model, start_string):
  num_generate = 400

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.2

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [29]:
inp = input('Type a starting string: ')
print(generate_text(model, inp))

Type a starting string: hello
hellow?

LARTIUS:
No, go
Mome thus made with furnish spicurds. Isacabrot?
No, I, brife;--what you know you here conque more your trke!
Seeing deaply gapes, if he match you be not madal pabriciel.

VICINIO:
Prefented, Licat,' apparel, and Villovere-time
Greamf and will, dramove free, I pray,--Do beseech you in?
In loateng made?

LADY ANNE:
What away! how for I selfawith! wh at most die,
Did morthman tha
